In [15]:
PROJECT_ID='jchavezar-demo'
REGION='US'
DATASET_NAME = 'bqmlga4'

In [16]:
!bq mk --location=$REGION --dataset $PROJECT_ID:$DATASET_NAME

BigQuery error in mk operation: Dataset 'jchavezar-demo:bqmlga4' already exists.


In [17]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [18]:
%%bigquery --project $PROJECT_ID

SELECT 
    *
FROM
  `firebase-public-project.analytics_153293282.events_*`
    
TABLESAMPLE SYSTEM (1 PERCENT)

Downloading: 100%|██████████| 50000/50000 [00:01<00:00, 26188.21rows/s]


,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,user_properties,user_first_touch_timestamp,user_ltv,device,geo,app_info,traffic_source,stream_id,platform,event_dimensions
0,20180716,1531797039530001,screen_view,"[{'key': 'firebase_previous_id', 'value': {'st...",1531796970451001,NaN,22,98627,None,64BB7DFAE772A77F70FB63373D0808F5,"[{'key': 'ad_frequency', 'value': {'string_val...",1516479005247000,None,"{'category': 'mobile', 'mobile_brand_name': 'n...","{'continent': 'Americas', 'country': 'United S...","{'id': 'com.labpixies.flood', 'version': '2.62...","{'name': None, 'medium': 'organic', 'source': ...",1051193346,ANDROID,None
1,20180716,1531796976451000,screen_view,"[{'key': 'firebase_screen_class', 'value': {'s...",1530676492667000,NaN,22,98627,None,64BB7DFAE772A77F70FB63373D0808F5,"[{'key': 'ad_frequency', 'value': {'string_val...",1516479005247000,None,"{'category': 'mobile', 'mobile_brand_name': 'n...","{'continent': 'Americas', 'country': 'United S...","{'id': 'com.labpixies.flood', 'version': '2.62...","{'name': None, 'medium': 'organic', 'source': ...",1051193346,ANDROID,None
2,20180716,1531796997156003,user_engagement,"[{'key': 'firebase_screen_class', 'value': {'s...",1530676496749003,NaN,22,98627,None,64BB7DFAE772A77F70FB63373D0808F5,"[{'key': 'ad_frequency', 'value': {'string_val...",1516479005247000,None,"{'category': 'mobile', 'mobile_brand_name': 'n...","{'continent': 'Americas', 'country': 'United S...","{'id': 'com.labpixies.flood', 'version': '2.62...","{'name': None, 'medium': 'organic', 'source': ...",1051193346,ANDROID,None
3,20180716,1531763688248024,ad_reward,"[{'key': 'type', 'value': {'string_value': 'St...",<NA>,NaN,131,620172,None,9D535D543FD07B8247DEADB4669FADE6,"[{'key': 'ad_frequency', 'value': {'string_val...",1464575838196000,None,"{'category': 'mobile', 'mobile_brand_name': 'n...","{'continent': 'Americas', 'country': 'United S...","{'id': 'com.labpixies.flood', 'version': '2.62...","{'name': '(direct)', 'medium': '(none)', 'sour...",1051193346,ANDROID,None
4,20180716,1531763696119034,use_extra_steps,"[{'key': 'item_name', 'value': {'string_value'...",1531621780350034,NaN,131,620172,None,9D535D543FD07B8247DEADB4669FADE6,"[{'key': 'ad_frequency', 'value': {'string_val...",1464575838196000,None,"{'category': 'mobile', 'mobile_brand_name': 'n...","{'continent': 'Americas', 'country': 'United S...","{'id': 'com.labpixies.flood', 'version': '2.62...","{'name': '(direct)', 'medium': '(none)', 'sour...",1051193346,ANDROID,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,20180716,1531786359838001,user_engagement,"[{'key': 'engagement_time_msec', 'value': {'st...",<NA>,NaN,1,204374,None,027806D19A2E8FC158705BF10ACED423,"[{'key': 'first_open_time', 'value': {'string_...",1531786278838000,None,"{'category': 'tablet', 'mobile_brand_name': 'n...","{'continent': 'Americas', 'country': 'United S...","{'id': 'com.google.flood2', 'version': '2.6.30...","{'name': '(direct)', 'medium': '(none)', 'sour...",1051193347,IOS,None
49996,20180716,1531786355334002,screen_view,"[{'key': 'firebase_screen_id', 'value': {'stri...",<NA>,NaN,1,204374,None,027806D19A2E8FC158705BF10ACED423,"[{'key': 'first_open_time', 'value': {'string_...",1531786278838000,None,"{'category': 'tablet', 'mobile_brand_name': 'n...","{'continent': 'Americas', 'country': 'United S...","{'id': 'com.google.flood2', 'version': '2.6.30...","{'name': '(direct)', 'medium': '(none)', 'sour...",1051193347,IOS,None
49997,20180716,1531786355982003,screen_view,"[{'key': 'firebase_screen_id', 'value': {'stri...",1531786278334003,NaN,1,204374,None,027806D19A2E8FC158705BF10ACED423,"[{'key': 'first_open_time', 'value': {'string_...",1531786278838000,None,"{'category': 'tablet', 'mobile_brand_name': 'n...","{'continent': 'Americas', 'country': 'United S...","{'id': 'com.google.flood2', 'version

In [19]:
%%bigquery --project $PROJECT_ID

SELECT 
    COUNT(DISTINCT user_pseudo_id) as count_distinct_users,
    COUNT(event_timestamp) as count_events
FROM
  `firebase-public-project.analytics_153293282.events_*`

Executing query with job ID: bb44c35a-b0f0-4859-bac7-e83f9474e88a
Query executing: 4.03s

KeyboardInterrupt: 

In [ ]:
%%bigquery --project $PROJECT_ID 

CREATE OR REPLACE VIEW bqmlga4.returningusers AS (
  WITH firstlasttouch AS (
    SELECT
      user_pseudo_id,
      MIN(event_timestamp) AS user_first_engagement,
      MAX(event_timestamp) AS user_last_engagement
    FROM
      `firebase-public-project.analytics_153293282.events_*`
    WHERE event_name="user_engagement"
    GROUP BY
      user_pseudo_id

  )
  SELECT
    user_pseudo_id,
    user_first_engagement,
    user_last_engagement,
    EXTRACT(MONTH from TIMESTAMP_MICROS(user_first_engagement)) as month,
    EXTRACT(DAYOFYEAR from TIMESTAMP_MICROS(user_first_engagement)) as julianday,
    EXTRACT(DAYOFWEEK from TIMESTAMP_MICROS(user_first_engagement)) as dayofweek,

    #add 24 hr to user's first touch
    (user_first_engagement + 86400000000) AS ts_24hr_after_first_engagement,

#churned = 1 if last_touch within 24 hr of app installation, else 0
IF (user_last_engagement < (user_first_engagement + 86400000000),
    1,
    0 ) AS churned,

#bounced = 1 if last_touch within 10 min, else 0
IF (user_last_engagement <= (user_first_engagement + 600000000),
    1,
    0 ) AS bounced,
  FROM
    firstlasttouch
  GROUP BY
    1,2,3
    );

SELECT 
  * 
FROM 
  bqmlga4.returningusers 
LIMIT 100;

Downloading: 100%|██████████| 100/100 [00:01<00:00, 67.44rows/s]


,user_pseudo_id,user_first_engagement,user_last_engagement,month,julianday,dayofweek,ts_24hr_after_first_engagement,churned,bounced
0,B390E09582A6DEF9545267A37C78B809,1529927874478004,1538025028130005,6,176,2,1530014274478004,0,0
1,023A2DB99CC3F70A4B71869C36952D74,1532347235020002,1536879053022028,7,204,2,1532433635020002,0,0
2,B99FD965B0874192EE94E8A8FE85EA9E,1532261629856004,1536856238641012,7,203,1,1532348029856004,0,0
3,F0186BDA8E7BF76675999CFDCFB10F6B,1536159830116007,1537105354822010,9,248,4,1536246230116007,0,0
4,01B358175084B5B44F1C83C2A42FE33A,1530307691225003,1538238216265003,6,180,6,1530394091225003,0,0
...,...,...,...,...,...,...,...,...,...
95,44561FB4558913850A7857FC521C1C42,1532903221480001,1537198045453043,7,210,1,1532989621480001,0,0
96,81B0637636FF876E195B4202F50CF215,1529983591678005,1537614681755042,6,177,3,1530069991678005,0,0
97,2D910124E99E4CBCD46F2D24E9F4EEA4,1533387488368005,1538332745082005,8,216,7,1533473888368005,0,0
98,27210AB7E69B78491EC3560089A193AD,1537186326283001,1537883841628009,9,260,2,1537272726283001,0,0


In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
    bounced,
    churned, 
    COUNT(churned) as count_users
FROM
    bqmlga4.returningusers
GROUP BY 1,2
ORDER BY bounced

Downloading: 100%|██████████| 3/3 [00:01<00:00,  2.21rows/s]


,bounced,churned,count_users
0,0,0,6148
1,0,1,1883
2,1,1,5557


In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
    COUNTIF(churned=1)/COUNT(churned) as churn_rate
FROM
    bqmlga4.returningusers
WHERE bounced = 0

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.36s/rows]


,churn_rate
0,0.234466


In [ ]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE VIEW bqmlga4.user_demographics AS (

  WITH first_values AS (
      SELECT
          user_pseudo_id,
          geo.country as country,
          device.operating_system as operating_system,
          device.language as language,
          ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp DESC) AS row_num
      FROM `firebase-public-project.analytics_153293282.events_*`
      WHERE event_name="user_engagement"
      )
  SELECT * EXCEPT (row_num)
  FROM first_values
  WHERE row_num = 1
  );

SELECT
  *
FROM
  bqmlga4.user_demographics
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  7.38rows/s]


,user_pseudo_id,country,operating_system,language
0,010DAA54F4C62559F2F5767813B48782,New Zealand,IOS,en-nz
1,018F03084C88E944163F76A0D0C2FBA1,Spain,ANDROID,de-de
2,04D9A2EAFCC291BF86F624008C55776C,Canada,IOS,en-ca
3,070F163E8E59260C722023967BB1C529,United States,ANDROID,en-us
4,08BBB3E2C4AF376339E1DA60FCF104B6,United States,IOS,en-us
5,09FF3AA4F3FE00E7FC084395C4615EA6,India,ANDROID,en-gb
6,0D9BFDCFA23E0715578EF21E93B6E794,United States,ANDROID,en-us
7,137944D29FEB9F2C96187C5F6604F5DB,Australia,IOS,en-au
8,16015784B69B59CAD1A4E556DA48ED50,Spain,IOS,es-es
9,171871CA6AF9A891FB7B3E19CD144B2E,India,ANDROID,en-us


In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
    event_name,
    COUNT(event_name) as event_count
FROM
    `firebase-public-project.analytics_153293282.events_*`
GROUP BY 1
ORDER BY
   event_count DESC

Downloading: 100%|██████████| 37/37 [00:01<00:00, 23.05rows/s]


,event_name,event_count
0,screen_view,2247623
1,user_engagement,1358958
2,level_start_quickplay,523430
3,level_end_quickplay,349729
4,post_score,242051
5,level_complete_quickplay,191088
6,level_fail_quickplay,137035
7,level_reset_quickplay,122278
8,select_content,105139
9,level_start,74417


In [ ]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE VIEW bqmlga4.user_aggregate_behavior AS (
WITH
  events_first24hr AS (
    #select user data only from first 24 hr of using the app
    SELECT
      e.*
    FROM
      `firebase-public-project.analytics_153293282.events_*` e
    JOIN
      bqmlga4.returningusers r
    ON
      e.user_pseudo_id = r.user_pseudo_id
    WHERE
      e.event_timestamp <= r.ts_24hr_after_first_engagement
    )
SELECT
  user_pseudo_id,
  SUM(IF(event_name = 'user_engagement', 1, 0)) AS cnt_user_engagement,
  SUM(IF(event_name = 'level_start_quickplay', 1, 0)) AS cnt_level_start_quickplay,
  SUM(IF(event_name = 'level_end_quickplay', 1, 0)) AS cnt_level_end_quickplay,
  SUM(IF(event_name = 'level_complete_quickplay', 1, 0)) AS cnt_level_complete_quickplay,
  SUM(IF(event_name = 'level_reset_quickplay', 1, 0)) AS cnt_level_reset_quickplay,
  SUM(IF(event_name = 'post_score', 1, 0)) AS cnt_post_score,
  SUM(IF(event_name = 'spend_virtual_currency', 1, 0)) AS cnt_spend_virtual_currency,
  SUM(IF(event_name = 'ad_reward', 1, 0)) AS cnt_ad_reward,
  SUM(IF(event_name = 'challenge_a_friend', 1, 0)) AS cnt_challenge_a_friend,
  SUM(IF(event_name = 'completed_5_levels', 1, 0)) AS cnt_completed_5_levels,
  SUM(IF(event_name = 'use_extra_steps', 1, 0)) AS cnt_use_extra_steps,
FROM
  events_first24hr
GROUP BY
  1
  );

SELECT
  *
FROM
  bqmlga4.user_aggregate_behavior
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  6.38rows/s]


,user_pseudo_id,cnt_user_engagement,cnt_level_start_quickplay,cnt_level_end_quickplay,cnt_level_complete_quickplay,cnt_level_reset_quickplay,cnt_post_score,cnt_spend_virtual_currency,cnt_ad_reward,cnt_challenge_a_friend,cnt_completed_5_levels,cnt_use_extra_steps
0,6CD6E64676EA73F0FC015F7DAF4A2DDC,26,0,0,0,0,6,0,0,0,1,0
1,747538074CEC9FE551F488F3E47F9FD9,54,11,3,0,5,2,1,0,0,0,1
2,264749B887AA92394A2F4E449D16A9F6,57,2,0,0,0,7,4,2,0,1,4
3,43A83473824B5860784FD7866EA79CE5,26,6,3,2,1,2,7,0,0,0,7
4,60BA5922F4F495D22AA8A87782F08517,11,4,3,3,0,3,0,0,0,0,0
5,78FE06A60B0758EEEEEE1B47FC59AF10,4,0,0,0,0,0,0,0,0,0,0
6,D4E7D53D818D952D21D3181CEA6BC289,9,3,3,2,0,2,0,0,0,0,0
7,5EE2F43F1FD31EAFBB6724FCBA21F203,12,3,3,0,0,0,0,0,0,0,0
8,23DB0DD049B22BF58416ED936D8EF270,25,7,6,5,0,5,5,0,0,0,5
9,A1483CA5D53030291E479D412CBA2397,2,0,0,0,0,0,0,0,0,0,0


In [ ]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE VIEW bqmlga4.train AS (
    
  SELECT
    dem.*,
    IFNULL(beh.cnt_user_engagement, 0) AS cnt_user_engagement,
    IFNULL(beh.cnt_level_start_quickplay, 0) AS cnt_level_start_quickplay,
    IFNULL(beh.cnt_level_end_quickplay, 0) AS cnt_level_end_quickplay,
    IFNULL(beh.cnt_level_complete_quickplay, 0) AS cnt_level_complete_quickplay,
    IFNULL(beh.cnt_level_reset_quickplay, 0) AS cnt_level_reset_quickplay,
    IFNULL(beh.cnt_post_score, 0) AS cnt_post_score,
    IFNULL(beh.cnt_spend_virtual_currency, 0) AS cnt_spend_virtual_currency,
    IFNULL(beh.cnt_ad_reward, 0) AS cnt_ad_reward,
    IFNULL(beh.cnt_challenge_a_friend, 0) AS cnt_challenge_a_friend,
    IFNULL(beh.cnt_completed_5_levels, 0) AS cnt_completed_5_levels,
    IFNULL(beh.cnt_use_extra_steps, 0) AS cnt_use_extra_steps,
    ret.user_first_engagement,
    ret.month,
    ret.julianday,
    ret.dayofweek,
    ret.churned
  FROM
    bqmlga4.returningusers ret
  LEFT OUTER JOIN
    bqmlga4.user_demographics dem
  ON 
    ret.user_pseudo_id = dem.user_pseudo_id
  LEFT OUTER JOIN 
    bqmlga4.user_aggregate_behavior beh
  ON
    ret.user_pseudo_id = beh.user_pseudo_id
  WHERE ret.bounced = 0
  );

SELECT
  *
FROM
  bqmlga4.train
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  7.01rows/s]


,user_pseudo_id,country,operating_system,language,cnt_user_engagement,cnt_level_start_quickplay,cnt_level_end_quickplay,cnt_level_complete_quickplay,cnt_level_reset_quickplay,cnt_post_score,cnt_spend_virtual_currency,cnt_ad_reward,cnt_challenge_a_friend,cnt_completed_5_levels,cnt_use_extra_steps,user_first_engagement,month,julianday,dayofweek,churned
0,94B02BA3020E382B3456932FA88A03AC,Canada,ANDROID,fr-ca,157,11,10,1,0,22,1,0,0,1,1,1536290454006001,9,250,6,0
1,7331597E5500811253BA85AF851A193C,United States,ANDROID,en-us,45,49,14,14,34,14,0,0,0,0,0,1535867140394001,9,245,1,0
2,FEDC59699D8832A11B88BFCCCAD0951D,United States,ANDROID,en-us,6,2,1,0,0,1,0,0,0,0,0,1529331681659005,6,169,2,0
3,6A8D61827CD1A7E5300673C05DEEDBCC,United States,ANDROID,en-us,8,4,3,2,1,2,0,0,0,0,0,1529267276057003,6,168,1,0
4,CA04708DA02378999ECB402481FF1AE4,United States,ANDROID,en-us,31,15,8,0,4,8,0,0,0,0,0,1529324838704004,6,169,2,0
5,E8A6F486759AB51EB8EB75B74FB6875B,United States,ANDROID,en-us,1,0,0,0,0,0,0,0,0,0,0,1531685286054003,7,196,1,0
6,21D29079D74D0CA82E5850EB9C86F820,United States,ANDROID,en-us,51,13,11,8,1,12,0,0,0,0,0,1532264346670001,7,203,1,0
7,ED0AF1301B6404476B420A990B98ABD7,United States,ANDROID,en-us,339,155,152,116,0,116,0,0,0,0,0,1536028364095004,9,247,3,0
8,389AE1E45F3EA1FB2083DF6129301E00,United States,ANDROID,en-us,2,0,0,0,0,0,0,0,0,0,0,1529800627466001,6,175,1,0
9,AB6D35BB8740F67100873630F0E4E227,Germany,ANDROID,de-de,23,1,0,0,0,6,0,0,0,1,0,1536477329005004,9,252,1,1


In [23]:
%%bigquery df

SELECT
  *
FROM
  bqmlga4.train
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  6.79rows/s]


In [31]:
MEAN_SD = {
    "julianday": (df['julianday'].mean(), df['julianday'].std())
}
MEAN_SD

{'julianday': (166.6, 3.40587727318528)}

## Create Model with BQML Adding it to Vertex AI Model Registry

In [50]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE MODEL bqmlga4.churn_logreg

OPTIONS(
  MODEL_TYPE="LOGISTIC_REG",
  INPUT_LABEL_COLS=["churned"],
  MODEL_REGISTRY='vertex_ai',
  VERTEX_AI_MODEL_VERSION_ALIASES=['logistic_reg', 'experiment']
) AS

SELECT
  *
FROM
  bqmlga4.train

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 3382.50query/s]                        


""


In [42]:
df.groupby(['user_pseudo_id']).sum()['churned']

user_pseudo_id
2F72E1976C24D8DFEA138BBCFD304BAC    0
55BFC405FFA33C0781DE73101011E45B    0
5C1F957882700804AA2D02552FA51DEC    0
6F993F2E30EB7021282D54B16C90717B    0
6FD55D5E530CE79A6AC9C09BB03DEA3A    0
804E81754B148AA8A9CFB52FA2DCC120    0
84CFFE14D7ACD4D84D423E3302B2E479    0
C67A23F5B815523945E529AAB6C53D8D    0
CE7404D5A8831F2DE46BD0A013A8ED64    0
F84699AFFC57D0F74319C93F99D8CBC4    0
Name: churned, dtype: Int64

In [45]:
df['churned'].unique()

<IntegerArray>
[0]
Length: 1, dtype: Int64

In [54]:
%%biqquery

EXPORT MODEL bqmlga4.churn_logreg
OPTIONS(URI = 'gs://vtx-models/bqml/')

Query complete after 0.00s: 100%|██████████| 2/2 [00:00<00:00, 1818.08query/s]                        


""


In [55]:
!gsutil ls gs://vtx-models/bqml

gs://vtx-models/bqml/
gs://vtx-models/bqml/saved_model.pb
gs://vtx-models/bqml/assets/
gs://vtx-models/bqml/variables/


In [56]:
!gsutil ls gs://mco-mm/churn

gs://mco-mm/churn/explanation_metadata.json
gs://mco-mm/churn/saved_model.pb
gs://mco-mm/churn/assets/
gs://mco-mm/churn/variables/
